In [ ]:
import pandas as pd

#load data 
df = pd.read_csv('/Users/eb2007/Library/CloudStorage/OneDrive-UniversityofCambridge/Documents/PhD/YBT.csv')

#inspect data 
print(df.head())

In [ ]:
#check for missing values

df.info()
df.isnull().sum()
df.columns

In [ ]:
#inspecting sus column

df['diagnosis_69_TEXT'].dropna().head()
df['diagnosis_69_TEXT'].notnull().sum()
df['diagnosis_69_TEXT'].dropna().unique()
df['diagnosis_69_TEXT'].dropna().value_counts()
df[['diagnosis_69_TEXT', 'diagnosis','diagnosis_yes_no']].dropna().head(10)


In [ ]:
#pprint all column names with their question text - wanna know what was before diagnosis_69
for col in df.columns:
    print(f"{df.loc[0, col]}: {col}")

In [ ]:
#RQ can eq, sq, aq be used to predict diagnosis?

#create target column
df['autism_diagnosis'] = (
    df['diagnosis']
    .fillna('') #replace missing values with empty string
    .str.lower()
    .str.contains('autism')
    .astype(int)
)

In [ ]:
#check missing values in key columns

aq_cols = [f'aq_{i}' for i in range(1, 11)]
sq_cols = [f'sq10_{i}' for i in range(1, 11)]
eq_cols = [f'eq10_{i}' for i in range(1,11)]
demo_cols = ['age', 'sex','gender', 'ethn', 'hand', 'country']

#target columns
target_col = 'autism_diagnosis' 

#all colums needed
all_cols = aq_cols + sq_cols + eq_cols + demo_cols + [target_col]

#check missing values in all columns
missing_counts = df[all_cols].isnull().sum()

print(missing_counts)